# Setup

In [ ]:
import plotly.express as px
import numpy as np
import pandas as pd
from nn_spikes import NeuralNetwork, batchTrain, test
import spike_tools, utilities
import pickle

In [ ]:
data = pd.read_csv('./datasources/spikes/training_data.csv')
spikeLocations = pd.read_csv('./datasources/spikes/training_spike_locations.csv', index_col=0)

In [ ]:
data_training, data_validation, spikeIndexes_training, spikeIndexes_validation = spike_tools.dataPreProcess(data, 
                                                                                                            spikeLocations=spikeLocations)

# Training

### Set up results dict

In [ ]:
results = utilities.createResultsRepo([500])

### Create and train NNs

In [ ]:
for hid in results.keys():
    
    nn = NeuralNetwork(input_nodes=101, 
                       hidden_nodes=int(hid), 
                       output_nodes=4, 
                       lr=0.1,
                       error_function='difference-squared')

    nn, trainingCurve, validationCurve = batchTrain(data_training=data_training,
                                                              data_validation=data_validation,
                                                              spikeIndexes_training=spikeIndexes_training, 
                                                              spikeIndexes_validation=spikeIndexes_validation, 
                                                              nn=nn,
                                                              epochs=30,
                                                              plotCurves=False)
    results[hid]['nn'] = nn
    results[hid]['trainingCurve'] = trainingCurve
    results[hid]['validationCurve'] = validationCurve

---

In [ ]:
data_training

In [ ]:
spike_tools.getAverageWaveforms(data_training, spikeIndexes_training, classToPlot=1)

# Load Submission Data

In [ ]:
data_submission = pd.read_csv('./datasources/spikes/submission_data.csv')
data_submission.columns = ['time (s)', 'signal']

In [ ]:
data_submission, predictedSpikeIndexes_submission = spike_tools.dataPreProcess(data_submission, 
                                                                               threshold=1.0, 
                                                                               submission=True, 
                                                                               waveformSignalType='signalSavgol')

In [ ]:
sample = data_submission.iloc[2500:50000, :]

In [ ]:
spike_tools.plotSpikes([sample['signalFiltered'], sample['signal']], [sample['predictedSpike']])

In [ ]:
data_submission

Extract wavs

In [ ]:
waveforms = data_submission[data_submission['predictedSpike']==True]['waveform']

Make predictions

In [ ]:
predictions = spike_tools.classifySpikesMLP(waveforms.iloc[1:], nn)

In [ ]:
predictions

---

In [ ]:
import plotly.graph_objects as go
signals = [data_submission['signal'][2000:8000], data_submission['signalFiltered'][2000:8000]]

In [ ]:
fig = go.Figure()

for signal in signals:
    fig.add_trace(go.Scatter(
        x=signal.index,
        y=signal,
        mode='lines',
        name=signal.name,
        opacity=0.5,
    ))
        
fig.show()

---

In [ ]:
data_submission

In [ ]:
data_training['signalFilteredBP2'] = spike_tools.bandPassFilter(data_training['signal'], order=2)
data_training['signalFilteredBP3'] = spike_tools.bandPassFilter(data_training['signal'], order=3)

In [ ]:
oo = 'savgol+BP'
data_submission[oo] = savgol_filter(data_submission['signal'], 17, 2)

In [ ]:
sample = data_submission.iloc[100000:200000]

In [ ]:
spike_tools.plotSpikes([sample['signal'], 
                        sample['signalSavgol'], 
                        sample['signalSavgolBP']], [sample['predictedSpike']])